# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will incur a charge of ~$3USD from OpenAI usage.

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Creating a Simple RAG Pipeline with LangChain v.0.2.0
  4. Task 4: Synthetic Dataset Generation for Evaluation using Ragas (Optional)

- 🤝 Breakout Room #2
  1. Task 1: Evaluating our Pipeline with Ragas
  2. Task 2: Testing OpenAI's Claim
  3. Task 3: Selecting an Advanced Retriever and Evaluating

> NOTE: This Notebook *does* contain a bonus challenge, outlined at the bottom of the notebook, which you can complete instead of the notebook for full marks on the assignment.

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://python.langchain.com/v0.2/docs/versions/v0_2/) of LangChain v0.2.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai langchain-qdrant

We'll also get the "star of the show" today, which is Ragas!

In [2]:
!pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [3]:
!pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.2.0

Building on what we've been learning, we'll be leveraging LangChain v0.2.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

- [`PyMuPDFLoader`](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyMuPDFLoader.html)

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

PDF_LINK = "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf"

loader = PyMuPDFLoader(PDF_LINK)

documents = loader.load_and_split()

In [6]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 1,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html#langchain-text-splitters-character-recursivecharactertextsplitter)

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

CHUNK_SIZE = 200
CHUNK_OVERLAP = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [8]:
len(documents)

1863

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

- [`OpenAIEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html#langchain-openai-embeddings-base-openaiembeddings)

> NOTE: We are purposefully using an older embedding model to try and answer the guiding question: Is TE3 better than Ada-002?

In [9]:
from langchain_openai import OpenAIEmbeddings

EMBEDDING_MODEL = "text-embedding-ada-002"

embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

- [`Qdrant`](https://api.python.langchain.com/en/latest/qdrant/langchain_qdrant.qdrant.QdrantVectorStore.html#langchain_qdrant.qdrant.QdrantVectorStore)

> NOTE: You'll need to provide the embedding dimension for Ada-002!

In [10]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

LOCATION = ":memory:"
COLLECTION_NAME = "PMarca Blogs"
VECTOR_SIZE = 1536

In [12]:
qdrant_client = QdrantClient(location=LOCATION)

qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE)
)

qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME,
    embedding=embeddings
)

qdrant_vector_store.add_documents(documents)

['ed8703e716024560ac9b2737d5060f59',
 '45e2615201574e449bb6478e452e036e',
 '57d28e16c46b4e66a76d95cc8e199eab',
 '52bbc4ac131a42f8b74993e06f6c18b4',
 '1b7610e1b79048bdbeba724f69b0337e',
 '3fa18f90b31040018b3f8d294e49a4c0',
 'dc99e58b0fb14fdb9d2ff411958e67b4',
 'ec2e060d9b7d4a449f7097e3ad533fe7',
 '20b492b378a64dc8951d16060facc413',
 '2711f05fc2994f57a5bcad84640b8f6f',
 '64637fd9556d43c2ad0b50c5cf82c6bd',
 'cf71ec05bb0142f8b1fa971de3dd592e',
 'c7c3c41249cf45e7b0f2f5681693340f',
 '8ac88ee6483c445386f561a869adaf52',
 '15ab0c5a36bd4b9c92d7bd9f357bc18a',
 'dde97cacc4bc4066918e85bd1245c11d',
 '4079f2022c5a4dc3aa30c85bc55465f2',
 'be56cc8539b34bcbacbebcda76774b32',
 'c51f0f5a04f94d6796dab272f3ae66fa',
 '1993ce7ede0640f6a7d68b06f43276a8',
 '901f6c9154d24506828c8fe53860d7dc',
 '26ccf0cdf27c4dcaac92f946e418add4',
 '12c938f69f3747a2afc1112ad4acdaf6',
 'ce219258e4764bc9a7dfb9764de820c4',
 '61862a71965f4995aee51fbdccc42e22',
 'ca8bd36e17f441a294938a3eb9266809',
 'ad33f827420e4248941620fb4637eeee',
 

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.  

LW:
1) HNSW for approximate nearest neighbor search
2) vector quantization to reduce the vector size and memory and speed search  
3) written in Rust

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [13]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [14]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [15]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change
have a reasonable chance at succeeding.
Second rule of thumb:
Once you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '9607985905f445ada448a721efc11d12', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:
In a rapidly changing Held like technology, the best place to
get experience when you’re starting out is in younger, high-
growth companies.' metadata={'source': 'https://d1lamhf6l6yk6d.cloud

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [16]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

/Users/ledger/anaconda3/envs/llmops-course/lib/python3.11/site-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/Users/ledger/anaconda3/envs/llmops-course/lib/python3.11/site-packages/langsmith/client.py:5515: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [17]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [19]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer any questions based solely on the context below. If the context \
doesn't provide the answer, just say "I don't know".

Question:
{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [20]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_template(template)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

Let's test it out!

In [21]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

A rule of thumb for selecting an industry to invest in is to ensure that the forces of change within that industry have a reasonable chance of succeeding. Additionally, once you have picked an industry, it is advised to get right to the center of it, where the great opportunities can be found.


In [22]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '0cf7676f0a824f479678522e768cc8d4', '_collection_name': 'PMarca Blogs'}, page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.'), Document(metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [23]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?  

LW: I'm not sure I understand the question :).  Different parameters than what?

In [24]:
len(eval_documents)

624

> NOTE: 🛑 Running this cell as presented will incur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step. **YOU CAN SKIP THIS STEP BY LOADING THE `.csv` DIRECTLY FROM OUR REPOSITORY.** 🛑

#### Optional: SDG for Evaluation

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

num_qa_pairs = 20 # You can reduce the number of QA pairs to 5 if you're experiencing rate-limiting issues

testset = generator.generate_with_langchain_docs(eval_documents, num_qa_pairs, distributions)
testset.to_pandas()

Let's look at the output and see what we can learn about it!

In [26]:
testset.test_data[0]

DataRow(question='How has Arnold Schwarzenegger achieved success as a movie star over the last 20 years?', contexts=['Arnold Schwarzenegger’s success as a movie star, politician, and\nbusinessman over the last 20 years.\nIf you have at any point in your life lived a relatively structured\nexistence — probably due to some kind of job with regular oZce\nhours, meetings, and the like — you will know that there is noth-\ning more liberating than looking at your calendar and seeing\nnothing but free time for weeks ahead to work on the most\nimportant things in whatever order you want.\nThis also gives you the best odds of maximizing Yow, which is a\nwhole ‘nother topic but highly related.\n128\nThe Pmarca Blog Archives'], ground_truth='The answer to given question is not present in context', evolution_type='simple', metadata=[{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/Th

In [27]:
testset_df = testset.to_pandas()
testset_df.to_csv("testset.csv")

#### PREFERRED: Download `.csv` from DataRepository

In [ ]:
!git clone https://github.com/AI-Maker-Space/DataRepository.git

Cloning into 'DataRepository'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 87 (delta 24), reused 28 (delta 8), pack-reused 8 (from 1)
Receiving objects: 100% (87/87), 70.09 MiB | 33.73 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [ ]:
!mv DataRepository/testset.csv .

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [28]:
import pandas as pd

test_df = pd.read_csv("testset.csv")

In [29]:
test_df

,Unnamed: 0,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,0,How has Arnold Schwarzenegger achieved success...,['Arnold Schwarzenegger’s success as a movie s...,The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,1,What types of undergraduate degrees are consid...,['workforce in a high-impact way when you grad...,Undergraduate degrees that are considered usef...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,2,How does the tendency to avoid inconsistency c...,['Five: Inconsistency-Avoidance Tendency\n[Peo...,The tendency to avoid inconsistency contribute...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,3,How has the presence of a billion people onlin...,['billion people online now. That is a very la...,The presence of a billion people online has si...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,4,What is the negative impact of a hyper-control...,['added humor value.]\nWhile I enjoyed Marc’s ...,The negative impact of a hyper-controlling man...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,5,How does the culture of a startup play a role ...,"['(In case you were wondering, by the way, the...",The culture of a startup plays a significant r...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,6,What are the consequences of not raising enoug...,['Here’s why you shouldn’t do that:\nWhat are ...,Not raising enough money risks the survival of...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,7,How can you prioritize building your product i...,['This obviously raises the issue of how you’r...,The most valuable thing you can do in the earl...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,8,How does the Influence-from-Mere-Association T...,['One very practical consequence of Liking/Lov...,The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,9,What is one important step for media companies...,['coherent message and strategy.\nThen go dark...,For media companies looking to turn around the...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [30]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [31]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [32]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [33]:
response_dataset[0]

{'question': 'How has Arnold Schwarzenegger achieved success as a movie star over the last 20 years?',
 'answer': "I don't know.",
 'contexts': ['This idea comes from a wonderful book called A Perfect Mess,\nwhich explains how not keeping a schedule has been key to\nArnold Schwarzenegger’s success as a movie star, politician, and',
  'done something, that you’ve accomplished something, that\nyou’re successful when a lot of other people weren’t.\nAnd of course, none of those things are true.',
  'you’ve been working hard your whole life, but working hard\nis not what I’m talking about. You’ve been continuously sur-\nrounded by a state of the art parental and educational support',
  'tion. (Hi, Tim! Hi, Diego!)\nNow, on the one hand, you can’t question the level of success of\neither company.\n70\nThe Pmarca Blog Archives'],
 'ground_truth': 'The answer to given question is not present in context'}

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [34]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [ ]:
results = evaluate(response_dataset, metrics)

In [36]:
results

{'faithfulness': 0.6544, 'answer_relevancy': 0.8642, 'context_recall': 0.7368, 'context_precision': 0.6886, 'answer_correctness': 0.6141}

In [37]:
results_df = results.to_pandas()
results_df

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How has Arnold Schwarzenegger achieved success...,[This idea comes from a wonderful book called ...,I don't know.,The answer to given question is not present in...,0.000000,0.000000,1.000000,0.000000,0.195204
1,What types of undergraduate degrees are consid...,[Which undergraduate degrees are useful in\nth...,Undergraduate degrees that are considered usef...,Undergraduate degrees that are considered usef...,1.000000,0.949308,1.000000,1.000000,0.666387
2,How does the tendency to avoid inconsistency c...,[less brain-blocked by its previous conclusion...,The tendency to avoid inconsistency contribute...,The tendency to avoid inconsistency contribute...,1.000000,0.969737,0.750000,1.000000,0.627840
3,How has the presence of a billion people onlin...,"[Which makes total sense, amid the enormous ma...",The presence of a billion people online has le...,The presence of a billion people online has si...,0.600000,0.901574,1.000000,0.916667,0.773790
4,What is the negative impact of a hyper-control...,[severe personality disorder who micromanages ...,The negative impact of a hyper-controlling man...,The negative impact of a hyper-controlling man...,1.000000,0.996238,0.750000,1.000000,0.573182
5,How does the culture of a startup play a role ...,"[(In case you were wondering, by the way, the ...",I don't know.,The culture of a startup plays a significant r...,0.000000,0.000000,0.333333,0.500000,0.180435
6,What are the consequences of not raising enoug...,[Here’s why you shouldn’t do that:\nWhat are t...,Not raising enough money risks the survival of...,Not raising enough money risks the survival of...,0.666667,0.961902,0.666667,0.833333,0.424897
7,How can you prioritize building your product i...,[whether your initial idea will work as a prod...,To prioritize building your product in the ear...,The most valuable thing you can do in the earl...,0.750000,0.970236,0.500000,0.500000,0.475144
8,How does the Influence-from-Mere-Association T...,[other facts to facilitate love.\nThe applicat...,The Influence-from-Mere-Association Tendency a...,The answer to given question is not present in...,0.600000,0.968781,1.000000,0.000000,0.176656
9,What is one important step for media companies...,[fucking easy.”\nAppendix for media companies ...,One important step for media companies to take...,For media companies looking to turn around the...,1.000000,1.000000,0.500000,1.000000,0.612682


## Task : Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #1:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

#### initializing embedding model

In [38]:
te3_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

#### creating a Qdrant collection, creating the datastore, and populating the datastore

In [39]:
qdrant_client.create_collection(
    collection_name=COLLECTION_NAME+"TE3",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME+"TE3",
    embedding=te3_embeddings,
)

qdrant_vector_store.add_documents(documents)

['eb006494d6ac4dd998727bafd1a3ea5a',
 'f851c91571394302a14c834b9bda15b6',
 '9e1635228a7e48a5aa6284beb8220c9c',
 '1d0b44152d104d06b4b95c8c787904c3',
 '0863d7114bab449ab81fb3c514dd11b1',
 '31418f95333344cd9685e12f64041eaa',
 '88a56ef2b8914db784abecaf2b6d5af9',
 '7cd29876786f4e9c8117c0ea0a0fd79a',
 'acf596677dad40cf9a923831852f0ab9',
 '9548ebae3ab84caa80816ec3d6808e96',
 '649769dc10944e5a977bca6591bd91c9',
 'df70030287a54fc69ddbe759ba53daaf',
 'de5d89af3d434552b4859f01132f721d',
 '6db256cbe5894822b4679b6cf02c09e2',
 'eaf6615319f349f0a39b4153751155a8',
 '9deca0a82df14e2cb3a635763e098919',
 '86fff089df0d40e78faa2d7181648c57',
 '6d9a12a114aa413b871a558f170fe45c',
 '2ceb7088236c4a129d50db98b17f1005',
 'c13795354268410191e61164c874c0c6',
 '26e53599931f423ebd87cdfb8471a691',
 '456d4863f630497087241953822b4c1e',
 'ca5f797cf69e49e0867887fd265437b0',
 '024b0fc14e144fb2af0db748490a1514',
 'f903c9d87f6c4e42bb00a456420d3ef2',
 'f5406479d95a4798b442f688a63ea689',
 'f10776166d4f4d64a908c3c8f83876d4',
 

#### getting the Qdrant Vectorstore retriever object for the above vectorstore

In [40]:
te3_retriever = qdrant_vector_store.as_retriever()

#### getting a langchain out-of-the-box qa doc stuffing chain

In [41]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

#### getting the langchain out-of-the-box RAG chain

In [42]:
from langchain.chains import create_retrieval_chain

te3_retrieval_chain = create_retrieval_chain(te3_retriever, document_chain)

#### collecting input responses to evaluate against our synthetic data

In [43]:
answers = []
contexts = []

for question in test_questions:
  response = te3_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

#### collect into a HF dataset of question-answer-context triplets with ground-truth labels

In [45]:
te3_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

#### evaluate with ragas

In [ ]:
te3_advanced_retrieval_results = evaluate(te3_response_dataset_advanced_retrieval, metrics)

In [47]:
te3_advanced_retrieval_results

{'faithfulness': 0.6295, 'answer_relevancy': 0.9684, 'context_recall': 0.7588, 'context_precision': 0.6579, 'answer_correctness': 0.6770}

#### collect results into Pandas DF for simple comparison

In [48]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(te3_advanced_retrieval_results.items()), columns=['Metric', 'TE3'])

df_merged = pd.merge(df_baseline, df_comparison, on='Metric')

df_merged['Baseline -> TE3'] = df_merged['TE3'] - df_merged['ADA']

df_merged

,Metric,ADA,TE3,Baseline -> TE3
0,faithfulness,0.654428,0.629522,-0.024906
1,answer_relevancy,0.864196,0.968434,0.104238
2,context_recall,0.736842,0.758772,0.021930
3,context_precision,0.688596,0.657895,-0.030702
4,answer_correctness,0.614074,0.676993,0.062919


####❓ Question #3:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?  

LW: I think we can safely say, yes, it is better, but not dramatically so, at least not from this one test.  I would want to do more testing across different domains to give a more confident answer.

## Task 5: Selecting an Advanced Retriever and Evaluating

#### 🏗️ Activity #2

While the changes that occured due to modifying the embedding model were desirable - you're now tasked with improving `context_recall`, or `context_precision` (or both!).

You'll follow these steps:

1. Reason about this list of Advanced Retrieval methods:
  - [Contextual Compression (Reranker)](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/contextual_compression/)
  - [MultiQueryRetriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/MultiQueryRetriever/)
  - [Parent Document Retriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/parent_document_retriever/)
2. Select the method you think will be the most performant.
3. Implement that method.
4. Create a LCEL chain that utlizes the new Retriever method.
5. Evaluate this LCEL and compare to the TE3 results.

> NOTE: We will spend more time in Session 14 diving into advanced retrieval methods, this activity is only to serve as a basic introduction to the idea of component-wise improvements and how they might impact metrics.

### Fetch and chunk documents  
- increasing the chunk size and overlap in an attempt to increase context-recall

In [55]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

cr_cp_documents = text_splitter_eval.split_documents(documents)

### Qdrant Vectorstore Retriever  
- increasing number of return documents to 7 in an attempt to improve context-recall (docs will be context compressed later)

In [56]:
COLLECTION_NAME = COLLECTION_NAME + '_CR_CP'

te3_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME,
    embedding=te3_embeddings
)

qdrant_vector_store.add_documents(documents)

In [60]:
cr_cp_retriever = qdrant_vector_store.as_retriever(k=7)

### Contextual Compression

In [61]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=cr_cp_retriever
)

#### Get LCEL RAG Chain

In [67]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

from langchain.prompts import ChatPromptTemplate

template = """
Answer any questions based solely on the context below. If the context \
doesn't provide the answer, just say "I don't know".

Question:
{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_template(template)

cr_cp_rag_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

In [69]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [84]:
answers = []
contexts = []

for question in test_questions:
  response = cr_cp_rag_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

In [16]:
from pydantic import BaseModel, TypeAdapter, ValidationError
from typing import List, Annotated
from typing_extensions import TypedDict

class RagasEvalDataset(TypedDict):
    question: Annotated[List[str], 'test questions']
    answer: Annotated[List[str], 'responses to evaluate']
    contexts: Annotated[List[str], 'contexts to evaluate']
    ground_truth: Annotated[List[str], 'test ground truth - labels']


ragas_eval_dataset = TypeAdapter(RagasEvalDataset)

In [ ]:
eval_data = RagasEvalDataset({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [ ]:
try:
    ragas_eval_dataset.validate_python(eval_data)
except ValidationError as e:
    print(e)

In [85]:
from datasets import Dataset

cr_cp_response_dataset_advanced_retrieval = Dataset.from_dict(eval_data)

In [86]:
cr_cp_advanced_retrieval_results = evaluate(cr_cp_response_dataset_advanced_retrieval, metrics)
cr_cp_advanced_retrieval_results

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

No statements were generated from the answer.


{'faithfulness': 0.8184, 'answer_relevancy': 0.8598, 'context_recall': 0.7675, 'context_precision': 0.7398, 'answer_correctness': 0.6666}

🎯 📊 We see an increase in context precision, likely due to the Contextual Compression step, as well as a slight bumb in context recall, which could be attributed to the larger chunk size, but may also just be not statitically significant.

In [89]:
df_baseline = pd.DataFrame(list(te3_advanced_retrieval_results.items()), columns=['Metric', 'TE3'])
df_comparison = pd.DataFrame(list(cr_cp_advanced_retrieval_results.items()), columns=['Metric', 'CR_CP'])

df_merged = pd.merge(df_baseline, df_comparison, on='Metric')

df_merged['TE3 -> CR_CP'] = df_merged['CR_CP'] - df_merged['TE3']

df_merged

,Metric,TE3,CR_CP,TE3 -> CR_CP
0,faithfulness,0.629522,0.818386,0.188865
1,answer_relevancy,0.968434,0.859758,-0.108677
2,context_recall,0.758772,0.767544,0.008772
3,context_precision,0.657895,0.739766,0.081871
4,answer_correctness,0.676993,0.666555,-0.010438


#### 🚧 BONUS CHALLENGE 🚧

> NOTE: Completing this challenge will provide full marks on the assignment, regardless of the complete of the notebook. You do not need to complete this in the notebook for full marks.

##### **MINIMUM REQUIREMENTS**:

1. Baseline `LCEL RAG` Application using `NAIVE RETRIEVAL`
2. Baseline Evaluation using `RAGAS METRICS`
  - [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
  - [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
  - [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
  - [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
  - [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)
3. Implement a `SEMANTIC CHUNKING STRATEGY`.
4. Create an `LCEL RAG` Application using `SEMANTIC CHUNKING` with `NAIVE RETRIEVAL`.
5. Compare and contrast results.

##### **SEMANTIC CHUNKING REQUIREMENTS**:

Chunk semantically similar (based on designed threshold) sentences, and then paragraphs, greedily, up to a maximum chunk size. Minimum chunk size is a single sentence.

Have fun!